In [1]:
import numpy as np

#### 1. Example UNIFAC Smith Van Ness fifth edition Appendix G

* Binary system diethylamine (1) /n-heptane (2)
* This calculation is done in molar fractions, for larger molecules like polymers of triacylglycerides, mass compositions must be used.

In [2]:
molar_x_1 = 0.4
molar_x_2 = 0.6

The subgroup data is organized in the following order:

1. k: subgroup identification number (This is an assigned number for the table)
2. Rk: Group area volume
3. Qk: Group area parameter
4. v^1_k: Number of times that group k is in the molecule 1
5. v^2_k: Number of times that group k is in the molecule 2

In [3]:
subgroup_data = {
    
   'CH3': [1, 0.9011, 0.848, 2, 2], 
   'CH2': [1, 0.6744, 0.540, 1, 5],  
   'CH2NH': [33, 1.2070, 0.936, 1, 0]
}

In [4]:
# r1,2,n.. = ri, is the molecular volume
r1 = subgroup_data['CH3'][3] * subgroup_data['CH3'][1] + subgroup_data['CH2'][3] * subgroup_data['CH2'][1] + subgroup_data['CH2NH'][3] * subgroup_data['CH2NH'][1]
r2 = subgroup_data['CH3'][4] * subgroup_data['CH3'][1] + subgroup_data['CH2'][4] * subgroup_data['CH2'][1] + subgroup_data['CH2NH'][4] * subgroup_data['CH2NH'][1]

In [5]:
# q1,2,n.. = qi, is the molecular area
q1 = subgroup_data['CH3'][3] * subgroup_data['CH3'][2] + subgroup_data['CH2'][3] * subgroup_data['CH2'][2] + subgroup_data['CH2NH'][3] * subgroup_data['CH2NH'][2]
q2 = subgroup_data['CH3'][4] * subgroup_data['CH3'][2] + subgroup_data['CH2'][4] * subgroup_data['CH2'][2] + subgroup_data['CH2NH'][4] * subgroup_data['CH2NH'][2]

##### 1.1 Combinatorial contribution

In [6]:
q_denominator = molar_x_1*q1 + molar_x_2*q2
r_denominator = molar_x_1*r1 + molar_x_2*r2

J1 = r1/r_denominator
L1 = q1/q_denominator

lngamma1c = 1 - J1 + np.log(J1) - 5*q1*(1-(J1/L1) + np.log(J1/L1))


J2 = r2/r_denominator
L2 = q2/q_denominator

lngamma2c = 1 - J2 + np.log(J2) - 5*q1*(1-(J2/L2) + np.log(J2/L2))

#### 1.2 Residual contribution

In [7]:
# eki, k is the subgroup, i is the molecule. eki is the fraction of superficial area that each subgroup adds to each molecule

e11 = subgroup_data['CH3'][3]*subgroup_data['CH3'][2]/q1
e12 = subgroup_data['CH3'][4]*subgroup_data['CH3'][2]/q2

e21 = subgroup_data['CH2'][3]*subgroup_data['CH2'][2]/q1
e22 = subgroup_data['CH2'][4]*subgroup_data['CH2'][2]/q2

e331 = subgroup_data['CH2NH'][3]*subgroup_data['CH2NH'][2]/q1
e332 = subgroup_data['CH2NH'][4]*subgroup_data['CH2NH'][2]/q1

eki = np.array([e11, e12, e21, e22, e331, e332])

In [8]:
# GIP from the literature, in k
a11 = 0
a12 = 0
a21 = 0
a22 = 0
a3333 = 0
a133 = 255.7
a233 = 255.7
a331 = 65.33
a332 = 65.33

GIP = np.array([a11, a12, a21, a22, a3333, a133, a233, a331, a332])

In [9]:
# Interaction energies
tao = np.exp(-GIP/308.15)

Bik = Summation(emi*taomk), m is a subindex that goes over all the subgroups, so if we take the subgroup k = 1 and component i = 1 we would get:

B1,1 = e1,1*tao1,1 + e2,1*tao2,1 + e33,1*tao33,1
B1,1 = 0.5347*1 + 0.1702*1 + 0.2951*0.8091

B1,1 = 0.9436

In [10]:
# Here we calculate the Bik
B11 = e11*tao[0] + e21*tao[2] + e331*tao[7]
B12 = e11*tao[1] + e21*tao[3] + e331*tao[8]
B133 = e11*tao[5] + e21*tao[6] + e331*tao[4]

B21 = e12*tao[0] + e22*tao[2] + e332*tao[7]
B22 = e12*tao[1] + e22*tao[3] + e332*tao[8]
B233 = e12*tao[5] + e22*tao[6] + e332*tao[4]

In [11]:
# Here we calcualte the thetak, j is a subindex that goes over all the molecules/components. This is an average effect of each subgroup in the overall area of the mixture.

theta1 = (molar_x_1*q1*e11 + molar_x_2*q2*e12 )/q_denominator

theta2 = (molar_x_1*q1*e21 + molar_x_2*q2*e22 )/q_denominator

theta3 = (molar_x_1*q1*e331 + molar_x_2*q2*e332 )/q_denominator


In [12]:
# Here we calculate the sk, weighted effect of each interaction energy with the average superficial area.

s1 = theta1*tao[0] + theta2*tao[2] + theta3*tao[7]

s2 = theta1*tao[1] + theta2*tao[3] + theta3*tao[8]

s33 = theta1*tao[5] + theta2*tao[6] + theta3*tao[4]

In [13]:
lngamma1r = q1*(1 - ((theta1*B11/s1 - e11*np.log(B11/s1)) + (theta2*B12/s2 - e21*np.log(B12/s2)) + (theta3*B133/s33 - e331*np.log(B133/s33))))

lngamma2r = q2*(1 - ((theta1*B21/s1 - e12*np.log(B21/s1)) + (theta2*B22/s2 - e22*np.log(B22/s2)) + (theta3*B233/s33 - e332*np.log(B233/s33))))

##### 1.3 Activity coefficient calculation

In [14]:
lngamma1 = lngamma1c + lngamma1r
gamma1 = np.exp(lngamma1)

lngamma2 = lngamma2c + lngamma2r
gamma2 = np.exp(lngamma2)
print(f"gamma 1: {gamma1}", f"gamma 2: {gamma2}")

gamma 1: 1.1330392999346754 gamma 2: 1.0469630934098826


#### 2. Biodiesel example

As an example of implementing the UNIFAC model to calculate activity coefficients, Experiment 23 from Table 1 in Reference 39 is selected, specifically the following tie line from Noriega et al. (2019). The composition is given in %w/w. The GIP and all additional parameters are also extracted from this paper.:


**Methanol_F:** 38.50     **FAME_F:** 38.50    **Glycerol_F:** 23.0  

**Methanol_L:** 11.6      **FAME_L:** 85.6     **Glycerol_L:** 2.8

**Methanol_H:** 32.3      **FAME_H:** 0.2265   **Glycerol_H:** 67.5



**The components are: Castor Oil Biodiesel (1), Methanol (2), Glycerol (3)** 

The fatty acid profile for the Oil is: (add 14 for the methyl ester and add 28 for ethyl ester in the molecular weight (MW))


**Palmitic:** 0.0074, MW: 256.43

**Stearic:** 0.0092, MW: 284.48

**Oleic:** 0.0271, MW: 282.46

**Linoleic:** 0.0431, MW: 280.4472

**Linolenic:** 0.0032, MW: 278.43

**Ricinolenic:** 0.8987, MW: 298.461

**Densipolic:** 0.0093, MW: 296.4


The groups are:

1. CH
2. CH=CH
3. OH
4. CH2COO
5. COOH
6. H2O

The order of the groups is mantained in the GIP Matrix as a 8x8 matrix with a diagonal of zeros.

##### 2.1 Additional data

In [15]:
GIP = np.array([
       [0,          0,          0,     -3214.25, -1718.09, -3079.37, 1295.55,  1249.74], 
       [0,          0,          0,     -3214.25, -1718.09, -3079.37, 1295.55,  1249.74],
       [0,          0,          0,     -3214.25, -1718.09, -3079.37, 1295.55,  1249.74],
       [2900.56,  2900.56,   2900.56,     0,     -388.62,  -964.53,  5084.03,  -987.18],
       [4925.67,  4925.67,   4925.67,   259.13,     0,      947.89, -1210.44,  -4618.51], 
       [2544.80,  2544.80,   2544.80,   1047.08, -2424.84,    0,    -685.81,    134.70], 
       [3714.86,  3714.86,   3714.86,   258.51,  -3227.03,  594.79,     0,      3239.57],
       [20930.91, 20930.91,  20930.91,  -688.85, -14448.32, 1158.19, -1429.48,     0]
       ])


TAO = np.exp(-GIP/298.15)

In [16]:
w1l = 0.856
w2l = 0.116
w3l = 0.028

w1h = 0.002265
w2h = 0.323
w3h = 0.675

In [17]:

# The order of this list is first is Rk (Volume), then Qk (Surface area), Molar mass 
group_parameters = {
    
    'CH': [0.4469, 0.2280, 13],
    'CH2': [0.6744, 0.5400, 14],
    'CH3': [0.9011, 0.8480, 15],
    'CH=CH': [1.1167, 0.8670, 26], 
    'OH': [1.000, 1.200, 17],
    'CH2COO': [1.6764, 1.4200, 58], #Grupo ester, es el grupo sin incluir el metilo/etilo y conectandose con el primer CH2 de la rama larga del acido graso.
    'H2O': [0.92, 1.40, 18],
    'COOH': [1.3013, 1.224, 45] #Grupo carboxilo, acido graso
}

In [18]:
FA_profile = {
    
    
"Palmitic": [0.0074,  256.43],

"Stearic": [0.0092, 284.48],

"Oleic": [0.0271, 282.46],

"Linoleic": [0.0431, 280.4472],

"Linolenic": [0.0032, 278.43],

"Ricinolenic": [0.8987, 298.461],

"Densipolic": [0.0093, 296.4]
    
}


n_inst = np.array([0, 0, 1, 2, 3, 1, 2])
p_OH = np.array([0, 0, 0, 0, 0, 1, 1])
m = np.array([14, 16, 14, 12, 10, 13, 11])

In [34]:
frac_acid_molar = []
frac_ester_molar = []
PM_acid = []
PM_ester = []
total_mol_acid = 0
total_mol_ester = 0


for i in FA_profile.values():
    
    mol_acid = i[0]/i[1]
    mol_ester = i[0]/(i[1] + 14)
    total_mol_acid += mol_acid 
    total_mol_ester += mol_ester
    
    PM_acid.append(i[1])
    frac_acid_molar.append(mol_acid)
    frac_ester_molar.append(mol_ester)

frac_acid_molar = np.array(frac_acid_molar)/total_mol_acid
frac_ester_molar = np.array(frac_ester_molar)/total_mol_ester
PM_acid = np.array(PM_acid) 

In [35]:


mean_n_acid = np.dot(frac_acid_molar, n_inst)

mean_p_acid = np.dot(frac_acid_molar, p_OH)

mean_m_acid = np.dot(frac_acid_molar, m)

print("Pseudomolecule parameters for the FA: \n" , f"n = {round(mean_n_acid, 4)} \n", f"p = {round(mean_p_acid, 4)} \n", f"m = {round(mean_m_acid, 4)} \n")

M_acid = np.dot(frac_acid_molar, PM_acid)
M_ester = (15+14*mean_m_acid+26*mean_n_acid+13*3*mean_p_acid+30*mean_p_acid+58+15)
M_OH = 32.04
M_Gly = 92.09382

Pseudomolecule parameters for the FA: 
 n = 1.0436 
 p = 0.9042 
 m = 12.9914 



**Biodiesel Molecule** 


CH3(CH2)m(CH=CH)n(CHOH)pCH2COO(CH2)bCH3

CH3(CH2)12.9913(CH=CH)1.0435(CHOH)0.9044CH2COO(CH2)0CH3

##### 2.2 Combinatorial contribution

The gamma is calculated for each component in each phase.

In [23]:
# Calculate r and q
r1 = 1/M_ester*(mean_p_acid*group_parameters['CH'][0] + mean_m_acid*group_parameters['CH2'][0] + 2*group_parameters['CH3'][0] + mean_n_acid*group_parameters['CH=CH'][0] + mean_p_acid*group_parameters['OH'][0]  + 1*group_parameters['CH2COO'][0]) # If CH3 is removed in both cases it doesn changes much
r2 = 1/M_OH*(1*group_parameters['CH3'][0] + 1*group_parameters['OH'][0])
r3 = 1/M_Gly*(1*group_parameters['CH'][0] + 2*group_parameters['CH2'][0] + 3*group_parameters['OH'][0])

q1 = 1/M_ester*(mean_p_acid*group_parameters['CH'][1] + mean_m_acid*group_parameters['CH2'][1] + 2*group_parameters['CH3'][1] + mean_n_acid*group_parameters['CH=CH'][1] + mean_p_acid*group_parameters['OH'][1] + 1*group_parameters['CH2COO'][1])
q2 = 1/M_OH*(1*group_parameters['CH3'][1] + 1*group_parameters['OH'][1])
q3 = 1/M_Gly*(1*group_parameters['CH'][1] + 2*group_parameters['CH2'][1] + 3*group_parameters['OH'][1])

# Light Phase
zeta_l = w1l/M_ester + w2l/M_OH + w3l/M_Gly
# Calculate phi and theta
phi1_l = r1*w1l/(r1*w1l + r2*w2l + r3*w3l)
phi2_l = r2*w2l/(r1*w1l + r2*w2l + r3*w3l)
phi3_l = r3*w3l/(r1*w1l + r2*w2l + r3*w3l)

theta1_l = q1*w1l/(q1*w1l + q2*w2l + q3*w3l)
theta2_l = q2*w2l/(q1*w1l + q2*w2l + q3*w3l)
theta3_l = q3*w3l/(q1*w1l + q2*w2l + q3*w3l)

#Calculate combinatorial
termA_1_l = zeta_l*M_ester*phi1_l/w1l
termB_1_l = phi1_l/theta1_l
termC_1_l = theta1_l/phi1_l
lngammac_l_1 = 1 - termA_1_l + np.log(termA_1_l) - 5*q1*M_ester*(1-(termB_1_l) - np.log(termC_1_l))

termA_2_l = zeta_l*M_OH*phi2_l/w2l
termB_2_l = phi2_l/theta2_l
termC_2_l = theta2_l/phi2_l
lngammac_l_2 = 1 - termA_2_l + np.log(termA_2_l) - 5*q2*M_OH*(1-(termB_2_l) - np.log(termC_2_l))

termA_3_l = zeta_l*M_Gly*phi3_l/w3l
termB_3_l = phi3_l/theta3_l
termC_3_l = theta3_l/phi3_l
lngammac_l_3 = 1 - termA_3_l + np.log(termA_3_l) - 5*q3*M_Gly*(1-(termB_3_l) - np.log(termC_3_l))

# Heavy Phase
zeta_h = w1h/M_ester + w2h/M_OH + w3h/M_Gly
# Calculate phi and theta
phi1_h = r1*w1h/(r1*w1h + r2*w2h + r3*w3h)
phi2_h = r2*w2h/(r1*w1h + r2*w2h + r3*w3h)
phi3_h = r3*w3h/(r1*w1h + r2*w2h + r3*w3h)

theta1_h = q1*w1h/(q1*w1h + q2*w2h + q3*w3h)
theta2_h = q2*w2h/(q1*w1h + q2*w2h + q3*w3h)
theta3_h = q3*w3h/(q1*w1h + q2*w2h + q3*w3h)

#Calculate combinatorial
termA_1_h = zeta_h*M_ester*phi1_h/w1h
termB_1_h = phi1_h/theta1_h
termC_1_h = theta1_h/phi1_h
lngammac_h_1 = 1 - termA_1_h + np.log(termA_1_h) - 5*q1*M_ester*(1-(termB_1_h) - np.log(termC_1_h))

termA_2_h = zeta_h*M_OH*phi2_h/w2h
termB_2_h = phi2_h/theta2_h
termC_2_h = theta2_h/phi2_h
lngammac_h_2 = 1 - termA_2_h + np.log(termA_2_h) - 5*q2*M_OH*(1-(termB_2_h) - np.log(termC_2_h))

termA_3_h = zeta_h*M_Gly*phi3_h/w3h
termB_3_h = phi3_h/theta3_h
termC_3_h = theta3_h/phi3_h
lngammac_h_3 = 1 - termA_3_h + np.log(termA_3_h) - 5*q3*M_Gly*(1-(termB_3_h) - np.log(termC_3_h))


##### 2.3 Residual contribution

In [24]:
# Light phase
corrected_w1l = w1l/(M_ester*zeta_l)
corrected_w2l = w2l/(M_OH*zeta_l)
corrected_w3l = w3l/(M_Gly*zeta_l)

corrected_CH_l = corrected_w1l*mean_p_acid + corrected_w3l*1
corrected_CH2_l = corrected_w1l*mean_m_acid + corrected_w3l*2
corrected_CH3_l = corrected_w1l*2 + corrected_w2l*1
corrected_CHCH_l = corrected_w1l*mean_n_acid
corrected_OH_l = corrected_w1l*mean_p_acid + corrected_w2l*1 + corrected_w3l*3
corrected_CH2COO_l = corrected_w1l*1
corrected_COOH_l = 0
corrected_H2O_l = 0

total_comp_group_contribution =  corrected_CH_l + corrected_CH2_l + corrected_CH3_l + corrected_CHCH_l + corrected_OH_l + corrected_CH2COO_l + corrected_H2O_l

WCH_l = corrected_CH_l/total_comp_group_contribution
WCH2_l = corrected_CH2_l/total_comp_group_contribution
WCH3_l = corrected_CH3_l/total_comp_group_contribution
WCHCH_l = corrected_CHCH_l/total_comp_group_contribution
WOH_l = corrected_OH_l/total_comp_group_contribution
WCH2COO_l = corrected_CH2COO_l/total_comp_group_contribution
WCOOH_l = corrected_COOH_l/total_comp_group_contribution
WH2O_l = corrected_H2O_l/total_comp_group_contribution

total_group_contribution = (group_parameters["CH"][1]*WCH_l + group_parameters["CH2"][1]*WCH2_l + group_parameters["CH3"][1]*WCH3_l + group_parameters["CH=CH"][1]*WCHCH_l + group_parameters["OH"][1]*WOH_l + group_parameters["CH2COO"][1]*WCH2COO_l + group_parameters["COOH"][1]*WCOOH_l + group_parameters["H2O"][1]*WH2O_l)

THETACH_l = group_parameters["CH"][1]*WCH_l/total_group_contribution
THETACH2_l = group_parameters["CH2"][1]*WCH2_l/total_group_contribution
THETACH3_l = group_parameters["CH3"][1]*WCH3_l/total_group_contribution
THETACHCH_l = group_parameters["CH=CH"][1]*WCHCH_l/total_group_contribution
THETAOH_l = group_parameters["OH"][1]*WOH_l/total_group_contribution
THETACH2COO_l = group_parameters["CH2COO"][1]*WCH2COO_l/total_group_contribution
THETACOOH_l = group_parameters["COOH"][1]*WCOOH_l/total_group_contribution
THETAH2O_l = group_parameters["H2O"][1]*WH2O_l/total_group_contribution

rTermACH_l = THETACH_l*TAO[0][0] + THETACH2_l*TAO[1][0] + THETACH3_l*TAO[2][0] +THETACHCH_l*TAO[3][0] + THETAOH_l*TAO[4][0] + THETACH2COO_l*TAO[5][0]+ THETACOOH_l*TAO[6][0]+ THETAH2O_l*TAO[7][0] 
rTermACH2_l = THETACH_l*TAO[0][1] + THETACH2_l*TAO[1][1] + THETACH3_l*TAO[2][1] +THETACHCH_l*TAO[3][1] + THETAOH_l*TAO[4][1] + THETACH2COO_l*TAO[5][1]+ THETACOOH_l*TAO[6][1]+ THETAH2O_l*TAO[7][1] 
rTermACH3_l = THETACH_l*TAO[0][2] +THETACH2_l*TAO[1][2] + THETACH3_l*TAO[2][2] +THETACHCH_l*TAO[3][2] + THETAOH_l*TAO[4][2] + THETACH2COO_l*TAO[5][2]+ THETACOOH_l*TAO[6][2]+ THETAH2O_l*TAO[7][2] 
rTermACHCH_l = THETACH_l*TAO[0][3] +THETACH2_l*TAO[1][3] + THETACH3_l*TAO[2][3] +THETACHCH_l*TAO[3][3] + THETAOH_l*TAO[4][3] + THETACH2COO_l*TAO[5][3]+ THETACOOH_l*TAO[6][3]+ THETAH2O_l*TAO[7][3] 
rTermAOH_l = THETACH_l*TAO[0][4] +THETACH2_l*TAO[1][4] + THETACH3_l*TAO[2][4] +THETACHCH_l*TAO[3][4] + THETAOH_l*TAO[4][4] + THETACH2COO_l*TAO[5][4]+ THETACOOH_l*TAO[6][4]+ THETAH2O_l*TAO[7][4] 
rTermACH2COO_l = THETACH_l*TAO[0][5] +THETACH2_l*TAO[1][5] + THETACH3_l*TAO[2][5] +THETACHCH_l*TAO[3][5] + THETAOH_l*TAO[4][5] + THETACH2COO_l*TAO[5][5]+ THETACOOH_l*TAO[6][5]+ THETAH2O_l*TAO[7][5] 
rTermACOOH_l = THETACH_l*TAO[0][6] +THETACH2_l*TAO[1][6] + THETACH3_l*TAO[2][6] +THETACHCH_l*TAO[3][6] + THETAOH_l*TAO[4][6] + THETACH2COO_l*TAO[5][6]+ THETACOOH_l*TAO[6][6]+ THETAH2O_l*TAO[7][6] 
rTermAH2O_l = THETACH_l*TAO[0][7] +THETACH2_l*TAO[1][7] + THETACH3_l*TAO[2][7] +THETACHCH_l*TAO[3][7] + THETAOH_l*TAO[4][7] + THETACH2COO_l*TAO[5][7]+ THETACOOH_l*TAO[6][7]+ THETAH2O_l*TAO[7][7] 


lngammar_CH_l = group_parameters["CH"][1]*(1 - np.log(rTermACH_l) - (THETACH_l*TAO[0][0]/rTermACH_l 
                                                                     + THETACH2_l*TAO[0][1]/rTermACH2_l 
                                                                     + THETACH3_l*TAO[0][2]/rTermACH3_l 
                                                                     + THETACHCH_l*TAO[0][3]/rTermACHCH_l 
                                                                     + THETAOH_l*TAO[0][4]/rTermAOH_l 
                                                                     + THETACH2COO_l*TAO[0][5]/rTermACH2COO_l 
                                                                     + THETACOOH_l*TAO[0][6]/rTermACOOH_l 
                                                                     + THETAH2O_l*TAO[0][7]/rTermAH2O_l))

lngammar_CH2_l = group_parameters["CH2"][1]*(1 - np.log(rTermACH2_l) - (THETACH_l*TAO[1][0]/rTermACH_l 
                                                                        + THETACH2_l*TAO[1][1]/rTermACH2_l 
                                                                        + THETACH3_l*TAO[1][2]/rTermACH3_l 
                                                                        + THETACHCH_l*TAO[1][3]/rTermACHCH_l 
                                                                        + THETAOH_l*TAO[1][4]/rTermAOH_l 
                                                                        + THETACH2COO_l*TAO[1][5]/rTermACH2COO_l 
                                                                        + THETACOOH_l*TAO[1][6]/rTermACOOH_l 
                                                                        + THETAH2O_l*TAO[1][7]/rTermAH2O_l))

lngammar_CH3_l = group_parameters["CH3"][1]*(1 - np.log(rTermACH3_l) - (THETACH_l*TAO[2][0]/rTermACH_l 
                                                                        + THETACH2_l*TAO[2][1]/rTermACH2_l 
                                                                        + THETACH3_l*TAO[2][2]/rTermACH3_l 
                                                                        + THETACHCH_l*TAO[2][3]/rTermACHCH_l 
                                                                        + THETAOH_l*TAO[2][4]/rTermAOH_l 
                                                                        + THETACH2COO_l*TAO[2][5]/rTermACH2COO_l 
                                                                        + THETACOOH_l*TAO[2][6]/rTermACOOH_l 
                                                                        + THETAH2O_l*TAO[2][7]/rTermAH2O_l))

lngammar_CHCH_l = group_parameters["CH=CH"][1]*(1 - np.log(rTermACHCH_l) - (THETACH_l*TAO[3][0]/rTermACH_l 
                                                                            + THETACH2_l*TAO[3][1]/rTermACH2_l 
                                                                            + THETACH3_l*TAO[3][2]/rTermACH3_l 
                                                                            + THETACHCH_l*TAO[3][3]/rTermACHCH_l 
                                                                            + THETAOH_l*TAO[3][4]/rTermAOH_l 
                                                                            + THETACH2COO_l*TAO[3][5]/rTermACH2COO_l 
                                                                            + THETACOOH_l*TAO[3][6]/rTermACOOH_l 
                                                                            + THETAH2O_l*TAO[3][7]/rTermAH2O_l))

lngammar_OH_l = group_parameters["OH"][1]*(1 - np.log(rTermAOH_l) - (THETACH_l*TAO[4][0]/rTermACH_l 
                                                                     + THETACH2_l*TAO[4][1]/rTermACH2_l 
                                                                     + THETACH3_l*TAO[4][2]/rTermACH3_l 
                                                                     + THETACHCH_l*TAO[4][3]/rTermACHCH_l 
                                                                     + THETAOH_l*TAO[4][4]/rTermAOH_l 
                                                                     + THETACH2COO_l*TAO[4][5]/rTermACH2COO_l 
                                                                     + THETACOOH_l*TAO[4][6]/rTermACOOH_l 
                                                                     + THETAH2O_l*TAO[4][7]/rTermAH2O_l))

lngammar_CH2COO_l = group_parameters["CH2COO"][1]*(1 - np.log(rTermACH2COO_l) - (THETACH_l*TAO[5][0]/rTermACH_l 
                                                                                 + THETACH2_l*TAO[5][1]/rTermACH2_l 
                                                                                 + THETACH3_l*TAO[5][2]/rTermACH3_l 
                                                                                 + THETACHCH_l*TAO[5][3]/rTermACHCH_l 
                                                                                 + THETAOH_l*TAO[5][4]/rTermAOH_l 
                                                                                 + THETACH2COO_l*TAO[5][5]/rTermACH2COO_l 
                                                                                 + THETACOOH_l*TAO[5][6]/rTermACOOH_l 
                                                                                 + THETAH2O_l*TAO[5][7]/rTermAH2O_l))

lngammar_COOH_l = group_parameters["COOH"][1]*(1 - np.log(rTermACOOH_l) - (THETACH_l*TAO[6][0]/rTermACH_l 
                                                                           + THETACH2_l*TAO[6][1]/rTermACH2_l 
                                                                           + THETACH3_l*TAO[6][2]/rTermACH3_l 
                                                                           + THETACHCH_l*TAO[6][3]/rTermACHCH_l 
                                                                           + THETAOH_l*TAO[6][4]/rTermAOH_l 
                                                                           + THETACH2COO_l*TAO[6][5]/rTermACH2COO_l 
                                                                           + THETACOOH_l*TAO[6][6]/rTermACOOH_l 
                                                                           + THETAH2O_l*TAO[6][7]/rTermAH2O_l))

lngammar_H2O_l = group_parameters["H2O"][1]*(1 - np.log(rTermAH2O_l) - (THETACH_l*TAO[7][0]/rTermACH_l 
                                                                        + THETACH2_l*TAO[7][1]/rTermACH2_l 
                                                                        + THETACH3_l*TAO[7][2]/rTermACH3_l 
                                                                        + THETACHCH_l*TAO[7][3]/rTermACHCH_l 
                                                                        + THETAOH_l*TAO[7][4]/rTermAOH_l 
                                                                        + THETACH2COO_l*TAO[7][5]/rTermACH2COO_l 
                                                                        + THETACOOH_l*TAO[7][6]/rTermACOOH_l 
                                                                        + THETAH2O_l*TAO[7][7]/rTermAH2O_l))

In [25]:
# heavy phase
corrected_w1h = w1h/(M_ester*zeta_h)
corrected_w2h = w2h/(M_OH*zeta_h)
corrected_w3h = w3h/(M_Gly*zeta_h)

corrected_CH_h = corrected_w1h*mean_p_acid + corrected_w3h*1
corrected_CH2_h = corrected_w1h*mean_m_acid + corrected_w3h*2
corrected_CH3_h = corrected_w1h*2 + corrected_w2h*1
corrected_CHCH_h = corrected_w1h*mean_n_acid
corrected_OH_h = corrected_w1h*mean_p_acid + corrected_w2h*1 + corrected_w3h*3
corrected_CH2COO_h = corrected_w1h*1
corrected_COOH_h = 0
corrected_H2O_h = 0

total_comp_group_contribution =  corrected_CH_h + corrected_CH2_h + corrected_CH3_h + corrected_CHCH_h + corrected_OH_h + corrected_CH2COO_h + corrected_H2O_h

WCH_h = corrected_CH_h/total_comp_group_contribution
WCH2_h = corrected_CH2_h/total_comp_group_contribution
WCH3_h = corrected_CH3_h/total_comp_group_contribution
WCHCH_h = corrected_CHCH_h/total_comp_group_contribution
WOH_h = corrected_OH_h/total_comp_group_contribution
WCH2COO_h = corrected_CH2COO_h/total_comp_group_contribution
WCOOH_h = corrected_COOH_h/total_comp_group_contribution
WH2O_h = corrected_H2O_h/total_comp_group_contribution

total_group_contribution = (group_parameters["CH"][1]*WCH_h + group_parameters["CH2"][1]*WCH2_h + group_parameters["CH3"][1]*WCH3_h + 
                            group_parameters["CH=CH"][1]*WCHCH_h + group_parameters["OH"][1]*WOH_h + group_parameters["CH2COO"][1]*WCH2COO_h + 
                            group_parameters["COOH"][1]*WCOOH_h + group_parameters["H2O"][1]*WH2O_h)

THETACH_h = group_parameters["CH"][1]*WCH_h/total_group_contribution
THETACH2_h = group_parameters["CH2"][1]*WCH2_h/total_group_contribution
THETACH3_h = group_parameters["CH3"][1]*WCH3_h/total_group_contribution
THETACHCH_h = group_parameters["CH=CH"][1]*WCHCH_h/total_group_contribution
THETAOH_h = group_parameters["OH"][1]*WOH_h/total_group_contribution
THETACH2COO_h = group_parameters["CH2COO"][1]*WCH2COO_h/total_group_contribution
THETACOOH_h = group_parameters["COOH"][1]*WCOOH_h/total_group_contribution
THETAH2O_h = group_parameters["H2O"][1]*WH2O_h/total_group_contribution

rTermACH_h = THETACH_h*TAO[0][0] + THETACH2_h*TAO[1][0] + THETACH3_h*TAO[2][0] +THETACHCH_h*TAO[3][0] + THETAOH_h*TAO[4][0] + THETACH2COO_h*TAO[5][0]+ THETACOOH_h*TAO[6][0]+ THETAH2O_h*TAO[7][0] 
rTermACH2_h = THETACH_h*TAO[0][1] + THETACH2_h*TAO[1][1] + THETACH3_h*TAO[2][1] +THETACHCH_h*TAO[3][1] + THETAOH_h*TAO[4][1] + THETACH2COO_h*TAO[5][1]+ THETACOOH_h*TAO[6][1]+ THETAH2O_h*TAO[7][1] 
rTermACH3_h = THETACH_h*TAO[0][2] +THETACH2_h*TAO[1][2] + THETACH3_h*TAO[2][2] +THETACHCH_h*TAO[3][2] + THETAOH_h*TAO[4][2] + THETACH2COO_h*TAO[5][2]+ THETACOOH_h*TAO[6][2]+ THETAH2O_h*TAO[7][2] 
rTermACHCH_h = THETACH_h*TAO[0][3] +THETACH2_h*TAO[1][3] + THETACH3_h*TAO[2][3] +THETACHCH_h*TAO[3][3] + THETAOH_h*TAO[4][3] + THETACH2COO_h*TAO[5][3]+ THETACOOH_h*TAO[6][3]+ THETAH2O_h*TAO[7][3] 
rTermAOH_h = THETACH_h*TAO[0][4] +THETACH2_h*TAO[1][4] + THETACH3_h*TAO[2][4] +THETACHCH_h*TAO[3][4] + THETAOH_h*TAO[4][4] + THETACH2COO_h*TAO[5][4]+ THETACOOH_h*TAO[6][4]+ THETAH2O_h*TAO[7][4] 
rTermACH2COO_h = THETACH_h*TAO[0][5] +THETACH2_h*TAO[1][5] + THETACH3_h*TAO[2][5] +THETACHCH_h*TAO[3][5] + THETAOH_h*TAO[4][5] + THETACH2COO_h*TAO[5][5]+ THETACOOH_h*TAO[6][5]+ THETAH2O_h*TAO[7][5] 
rTermACOOH_h = THETACH_h*TAO[0][6] +THETACH2_h*TAO[1][6] + THETACH3_h*TAO[2][6] +THETACHCH_h*TAO[3][6] + THETAOH_h*TAO[4][6] + THETACH2COO_h*TAO[5][6]+ THETACOOH_h*TAO[6][6]+ THETAH2O_h*TAO[7][6] 
rTermAH2O_h = THETACH_h*TAO[0][7] +THETACH2_h*TAO[1][7] + THETACH3_h*TAO[2][7] +THETACHCH_h*TAO[3][7] + THETAOH_h*TAO[4][7] + THETACH2COO_h*TAO[5][7]+ THETACOOH_h*TAO[6][7]+ THETAH2O_h*TAO[7][7] 

lngammar_CH_h = group_parameters["CH"][1]*(1 - np.log(rTermACH_h) - (THETACH_h*TAO[0][0]/rTermACH_h 
                                                                     + THETACH2_h*TAO[0][1]/rTermACH2_h 
                                                                     + THETACH3_h*TAO[0][2]/rTermACH3_h 
                                                                     + THETACHCH_h*TAO[0][3]/rTermACHCH_h 
                                                                     + THETAOH_h*TAO[0][4]/rTermAOH_h 
                                                                     + THETACH2COO_h*TAO[0][5]/rTermACH2COO_h 
                                                                     + THETACOOH_h*TAO[0][6]/rTermACOOH_h 
                                                                     + THETAH2O_h*TAO[0][7]/rTermAH2O_h))

lngammar_CH2_h = group_parameters["CH2"][1]*(1 - np.log(rTermACH2_h) - (THETACH_h*TAO[1][0]/rTermACH_h 
                                                                        + THETACH2_h*TAO[1][1]/rTermACH2_h 
                                                                        + THETACH3_h*TAO[1][2]/rTermACH3_h 
                                                                        + THETACHCH_h*TAO[1][3]/rTermACHCH_h 
                                                                        + THETAOH_h*TAO[1][4]/rTermAOH_h 
                                                                        + THETACH2COO_h*TAO[1][5]/rTermACH2COO_h 
                                                                        + THETACOOH_h*TAO[1][6]/rTermACOOH_h 
                                                                        + THETAH2O_h*TAO[1][7]/rTermAH2O_h))

lngammar_CH3_h = group_parameters["CH3"][1]*(1 - np.log(rTermACH3_h) - (THETACH_h*TAO[2][0]/rTermACH_h 
                                                                        + THETACH2_h*TAO[2][1]/rTermACH2_h 
                                                                        + THETACH3_h*TAO[2][2]/rTermACH3_h 
                                                                        + THETACHCH_h*TAO[2][3]/rTermACHCH_h 
                                                                        + THETAOH_h*TAO[2][4]/rTermAOH_h 
                                                                        + THETACH2COO_h*TAO[2][5]/rTermACH2COO_h 
                                                                        + THETACOOH_h*TAO[2][6]/rTermACOOH_h 
                                                                        + THETAH2O_h*TAO[2][7]/rTermAH2O_h))

lngammar_CHCH_h = group_parameters["CH=CH"][1]*(1 - np.log(rTermACHCH_h) - (THETACH_h*TAO[3][0]/rTermACH_h 
                                                                            + THETACH2_h*TAO[3][1]/rTermACH2_h 
                                                                            + THETACH3_h*TAO[3][2]/rTermACH3_h 
                                                                            + THETACHCH_h*TAO[3][3]/rTermACHCH_h 
                                                                            + THETAOH_h*TAO[3][4]/rTermAOH_h 
                                                                            + THETACH2COO_h*TAO[3][5]/rTermACH2COO_h 
                                                                            + THETACOOH_h*TAO[3][6]/rTermACOOH_h 
                                                                            + THETAH2O_h*TAO[3][7]/rTermAH2O_h))

lngammar_OH_h = group_parameters["OH"][1]*(1 - np.log(rTermAOH_h) - (THETACH_h*TAO[4][0]/rTermACH_h 
                                                                     + THETACH2_h*TAO[4][1]/rTermACH2_h 
                                                                     + THETACH3_h*TAO[4][2]/rTermACH3_h 
                                                                     + THETACHCH_h*TAO[4][3]/rTermACHCH_h 
                                                                     + THETAOH_h*TAO[4][4]/rTermAOH_h 
                                                                     + THETACH2COO_h*TAO[4][5]/rTermACH2COO_h 
                                                                     + THETACOOH_h*TAO[4][6]/rTermACOOH_h 
                                                                     + THETAH2O_h*TAO[4][7]/rTermAH2O_h))

lngammar_CH2COO_h = group_parameters["CH2COO"][1]*(1 - np.log(rTermACH2COO_h) - (THETACH_h*TAO[5][0]/rTermACH_h 
                                                                                 + THETACH2_h*TAO[5][1]/rTermACH2_h 
                                                                                 + THETACH3_h*TAO[5][2]/rTermACH3_h 
                                                                                 + THETACHCH_h*TAO[5][3]/rTermACHCH_h 
                                                                                 + THETAOH_h*TAO[5][4]/rTermAOH_h 
                                                                                 + THETACH2COO_h*TAO[5][5]/rTermACH2COO_h 
                                                                                 + THETACOOH_h*TAO[5][6]/rTermACOOH_h 
                                                                                 + THETAH2O_h*TAO[5][7]/rTermAH2O_h))

lngammar_COOH_h = group_parameters["COOH"][1]*(1 - np.log(rTermACOOH_h) - (THETACH_h*TAO[6][0]/rTermACH_h 
                                                                           + THETACH2_h*TAO[6][1]/rTermACH2_h 
                                                                           + THETACH3_h*TAO[6][2]/rTermACH3_h 
                                                                           + THETACHCH_h*TAO[6][3]/rTermACHCH_h 
                                                                           + THETAOH_h*TAO[6][4]/rTermAOH_h 
                                                                           + THETACH2COO_h*TAO[6][5]/rTermACH2COO_h 
                                                                           + THETACOOH_h*TAO[6][6]/rTermACOOH_h 
                                                                           + THETAH2O_h*TAO[6][7]/rTermAH2O_h))

lngammar_H2O_h = group_parameters["H2O"][1]*(1 - np.log(rTermAH2O_h) - (THETACH_h*TAO[7][0]/rTermACH_h 
                                                                        + THETACH2_h*TAO[7][1]/rTermACH2_h 
                                                                        + THETACH3_h*TAO[7][2]/rTermACH3_h 
                                                                        + THETACHCH_h*TAO[7][3]/rTermACHCH_h 
                                                                        + THETAOH_h*TAO[7][4]/rTermAOH_h 
                                                                        + THETACH2COO_h*TAO[7][5]/rTermACH2COO_h 
                                                                        + THETACOOH_h*TAO[7][6]/rTermACOOH_h 
                                                                        + THETAH2O_h*TAO[7][7]/rTermAH2O_h))


In [26]:
# Pure 1
total_comp_group_contribution = (2*mean_p_acid + mean_m_acid + 2 + mean_n_acid + 1)
WCH_1 = (mean_p_acid)/total_comp_group_contribution
WCH2_1 = (mean_m_acid)/total_comp_group_contribution
WCH3_1 = (2)/total_comp_group_contribution
WCHCH_1 = (mean_n_acid)/total_comp_group_contribution
WOH_1 = (mean_p_acid)/total_comp_group_contribution
WCH2COO_1 = (1)/total_comp_group_contribution
total_group_contribution = (group_parameters["CH"][1]*WCH_1 + group_parameters["CH2"][1]*WCH2_1 + group_parameters["CH3"][1]*WCH3_1 + group_parameters["CH=CH"][1]*WCHCH_1 + group_parameters["OH"][1]*WOH_1 + group_parameters["CH2COO"][1]*WCH2COO_1)

THETACH_1 = group_parameters["CH"][1]*WCH_1/total_group_contribution
THETACH2_1 = group_parameters["CH2"][1]*WCH2_1/total_group_contribution
THETACH3_1 = group_parameters["CH3"][1]*WCH3_1/total_group_contribution
THETACHCH_1 = group_parameters["CH=CH"][1]*WCHCH_1/total_group_contribution
THETAOH_1 = group_parameters["OH"][1]*WOH_1/total_group_contribution
THETACH2COO_1 = group_parameters["CH2COO"][1]*WCH2COO_1/total_group_contribution


rTermACH_1 = THETACH_1*TAO[0][0] + THETACH2_1*TAO[1][0] + THETACH3_1*TAO[2][0] +THETACHCH_1*TAO[3][0] + THETAOH_1*TAO[4][0] + THETACH2COO_1*TAO[5][0] 
rTermACH2_1 = THETACH_1*TAO[0][1] + THETACH2_1*TAO[1][1] + THETACH3_1*TAO[2][1] +THETACHCH_1*TAO[3][1] + THETAOH_1*TAO[4][1] + THETACH2COO_1*TAO[5][1] 
rTermACH3_1 = THETACH_1*TAO[0][2] +THETACH2_1*TAO[1][2] + THETACH3_1*TAO[2][2] +THETACHCH_1*TAO[3][2] + THETAOH_1*TAO[4][2] + THETACH2COO_1*TAO[5][2]  
rTermACHCH_1 = THETACH_1*TAO[0][3] +THETACH2_1*TAO[1][3] + THETACH3_1*TAO[2][3] +THETACHCH_1*TAO[3][3] + THETAOH_1*TAO[4][3] + THETACH2COO_1*TAO[5][3] 
rTermAOH_1 = THETACH_1*TAO[0][4] +THETACH2_1*TAO[1][4] + THETACH3_1*TAO[2][4] +THETACHCH_1*TAO[3][4] + THETAOH_1*TAO[4][4] + THETACH2COO_1*TAO[5][4] 
rTermACH2COO_1 = THETACH_1*TAO[0][5] +THETACH2_1*TAO[1][5] + THETACH3_1*TAO[2][5] +THETACHCH_1*TAO[3][5] + THETAOH_1*TAO[4][5] + THETACH2COO_1*TAO[5][5] 

lngammar_CH_1 = group_parameters["CH"][1]*(1 - np.log(rTermACH_1) - (THETACH_1*TAO[0][0]/rTermACH_1
                                                                                           + THETACH2_1*TAO[0][1]/rTermACH2_1
                                                                                           + THETACH3_1*TAO[0][2]/rTermACH3_1
                                                                                           + THETACHCH_1*TAO[0][3]/rTermACHCH_1 
                                                                                           + THETAOH_1*TAO[0][4]/rTermAOH_1
                                                                                           + THETACH2COO_1*TAO[0][5]/rTermACH2COO_1 
                                                                                           ))


lngammar_CH2_1 = group_parameters["CH2"][1]*(1 - np.log(rTermACH2_1) - (THETACH_1*TAO[1][0]/rTermACH_1
                                                                                           + THETACH2_1*TAO[1][1]/rTermACH2_1
                                                                                           + THETACH3_1*TAO[1][2]/rTermACH3_1
                                                                                           + THETACHCH_1*TAO[1][3]/rTermACHCH_1
                                                                                           + THETAOH_1*TAO[1][4]/rTermAOH_1
                                                                                           + THETACH2COO_1*TAO[1][5]/rTermACH2COO_1
                                                                                          ))



lngammar_CH3_1 = group_parameters["CH3"][1]*(1 - np.log(rTermACH3_1) - (THETACH_1*TAO[2][0]/rTermACH_1
                                                                                           + THETACH2_1*TAO[2][1]/rTermACH2_1
                                                                                           + THETACH3_1*TAO[2][2]/rTermACH3_1
                                                                                           + THETACHCH_1*TAO[2][3]/rTermACHCH_1 
                                                                                           + THETAOH_1*TAO[2][4]/rTermAOH_1
                                                                                           + THETACH2COO_1*TAO[2][5]/rTermACH2COO_1
                                                                                         ))


lngammar_CHCH_1 = group_parameters["CH=CH"][1]*(1 - np.log(rTermACHCH_1) - (THETACH_1*TAO[3][0]/rTermACH_1
                                                                                           + THETACH2_1*TAO[3][1]/rTermACH2_1
                                                                                           + THETACH3_1*TAO[3][2]/rTermACH3_1
                                                                                           + THETACHCH_1*TAO[3][3]/rTermACHCH_1 
                                                                                           + THETAOH_1*TAO[3][4]/rTermAOH_1
                                                                                           + THETACH2COO_1*TAO[3][5]/rTermACH2COO_1
                                                                                           ))



lngammar_OH_1 = group_parameters["OH"][1]*(1 - np.log(rTermAOH_1) - (THETACH_1*TAO[4][0]/rTermACH_1
                                                                                           + THETACH2_1*TAO[4][1]/rTermACH2_1
                                                                                           + THETACH3_1*TAO[4][2]/rTermACH3_1
                                                                                           + THETACHCH_1*TAO[4][3]/rTermACHCH_1 
                                                                                           + THETAOH_1*TAO[4][4]/rTermAOH_1
                                                                                           + THETACH2COO_1*TAO[4][5]/rTermACH2COO_1
                                                                                          ))

lngammar_CH2COO_1 = group_parameters["CH2COO"][1]*(1 - np.log(rTermACH2COO_1) - (THETACH_1*TAO[5][0]/rTermACH_1
                                                                                           + THETACH2_1*TAO[5][1]/rTermACH2_1
                                                                                           + THETACH3_1*TAO[5][2]/rTermACH3_1
                                                                                           + THETACHCH_1*TAO[5][3]/rTermACHCH_1 
                                                                                           + THETAOH_1*TAO[5][4]/rTermAOH_1
                                                                                           + THETACH2COO_1*TAO[5][5]/rTermACH2COO_1
                                                                                           ))


In [27]:
# Pure 2
total_comp_group_contribution = (2)
WCH3_2 = (1)/total_comp_group_contribution
WOH_2 = (1)/total_comp_group_contribution

total_group_contribution = ( group_parameters["CH3"][1]*WCH3_2  + group_parameters["OH"][1]*WOH_2)

THETACH3_2 = group_parameters["CH3"][1]*WCH3_2/total_group_contribution
THETAOH_2 = group_parameters["OH"][1]*WOH_2/total_group_contribution



rTermACH3_2 = THETACH3_2*TAO[2][2]  + THETAOH_2*TAO[4][2]  
rTermAOH_2 = THETACH3_2*TAO[2][4] + THETAOH_2*TAO[4][4]  

lngammar_CH3_2 = group_parameters["CH3"][1]*(1 - np.log(rTermACH3_2) - (THETACH3_2*TAO[2][2]/rTermACH3_2
                                                                                           + THETAOH_2*TAO[2][4]/rTermAOH_2))

lngammar_OH_2 = group_parameters["OH"][1]*(1 - np.log(rTermAOH_2) -(THETACH3_2*TAO[4][2]/rTermACH3_2
                                                                                           + THETAOH_2*TAO[4][4]/rTermAOH_2))


In [28]:
# Pure 3
total_comp_group_contribution = (6)
WCH_3 = (1)/total_comp_group_contribution
WCH2_3 = (2)/total_comp_group_contribution
WOH_3 = (3)/total_comp_group_contribution

total_group_contribution = (group_parameters["CH"][1]*WCH_3 + group_parameters["CH2"][1]*WCH2_3 + group_parameters["OH"][1]*WOH_3)

THETACH_3 = group_parameters["CH"][1]*WCH_3/total_group_contribution
THETACH2_3 = group_parameters["CH2"][1]*WCH2_3/total_group_contribution
THETAOH_3 = group_parameters["OH"][1]*WOH_3/total_group_contribution

rTermACH_3 = THETACH_3*TAO[0][0] + THETACH2_3*TAO[1][0] + THETAOH_3*TAO[4][0] 
rTermACH2_3 = THETACH_3*TAO[0][1] + THETACH2_3*TAO[1][1] + THETAOH_3*TAO[4][1] 
rTermAOH_3 = THETACH_3*TAO[0][4] +THETACH2_3*TAO[1][4] + THETAOH_3*TAO[4][4]  

lngammar_CH_3 = group_parameters["CH"][1]*(1 - np.log(rTermACH_3) - (THETACH_3*TAO[0][0]/rTermACH_3
                                                                                           + THETACH2_3*TAO[0][1]/rTermACH2_3 
                                                                                           + THETAOH_3*TAO[0][4]/rTermAOH_3))


lngammar_CH2_3 = group_parameters["CH2"][1]*(1 - np.log(rTermACH2_3) - (THETACH_3*TAO[1][0]/rTermACH_3
                                                                                           + THETACH2_3*TAO[1][1]/rTermACH2_3 
                                                                                           + THETAOH_3*TAO[1][4]/rTermAOH_3))

lngammar_OH_3 = group_parameters["OH"][1]*(1 - np.log(rTermAOH_3) - (THETACH_3*TAO[4][0]/rTermACH_3
                                                                                           + THETACH2_3*TAO[4][1]/rTermACH2_3 
                                                                                           + THETAOH_3*TAO[4][4]/rTermAOH_3))

In [29]:
lngammar_1_l = (mean_p_acid*(lngammar_CH_l - lngammar_CH_1) 
                + mean_m_acid*(lngammar_CH2_l - lngammar_CH2_1) 
                + 2*(lngammar_CH3_l - lngammar_CH3_1)
                + mean_n_acid*(lngammar_CHCH_l - lngammar_CHCH_1)
                + mean_p_acid*(lngammar_OH_l - lngammar_OH_1)
                + 1*(lngammar_CH2COO_l - lngammar_CH2COO_1))

lngammar_2_l = ((lngammar_CH3_l - lngammar_CH3_2)
                + 1*(lngammar_OH_l - lngammar_OH_2))

lngammar_3_l = ((lngammar_CH_l - lngammar_CH_3)
                + 2*(lngammar_CH2_l - lngammar_CH2_3)
                + 3*(lngammar_OH_l - lngammar_OH_3))

In [30]:
lngammar_1_h = (mean_p_acid*(lngammar_CH_h - lngammar_CH_1) 
                + mean_m_acid*(lngammar_CH2_h - lngammar_CH2_1) 
                + 2*(lngammar_CH3_h - lngammar_CH3_1)
                + mean_n_acid*(lngammar_CHCH_h - lngammar_CHCH_1)
                + mean_p_acid*(lngammar_OH_h - lngammar_OH_1)
                + 1*(lngammar_CH2COO_h - lngammar_CH2COO_1))

lngammar_2_h = ((lngammar_CH3_h - lngammar_CH3_2)
                + 1*(lngammar_OH_h - lngammar_OH_2))

lngammar_3_h = (1*(lngammar_CH_h - lngammar_CH_3)
                + 2*(lngammar_CH2_h - lngammar_CH2_3)
                + 3*(lngammar_OH_h - lngammar_OH_3))

In [31]:
lngamma_1_l = lngammac_l_1 + lngammar_1_l
lngamma_1_h = lngammac_h_1 + lngammar_1_h

lngamma_2_l = lngammac_l_2 + lngammar_2_l
lngamma_2_h = lngammac_h_2 + lngammar_2_h

lngamma_3_l = lngammac_l_3 + lngammar_3_l
lngamma_3_h = lngammac_h_3 + lngammar_3_h


In [32]:
gamma_1_l = np.exp(lngamma_1_l)
gamma_1_h = np.exp(lngamma_1_h)

gamma_2_l = np.exp(lngamma_2_l)
gamma_2_h = np.exp(lngamma_2_h)

gamma_3_l = np.exp(lngamma_3_l)
gamma_3_h = np.exp(lngamma_3_h)

In [33]:
print(gamma_1_l, gamma_1_h)

print(gamma_2_l, gamma_2_h)

print(gamma_3_l, gamma_3_h)

0.5721531425804245 1.3530438565501377e-16
0.2320066515212852 0.8454431132435113
0.014564142561568613 0.8312918462318348
